<a href="https://colab.research.google.com/github/gimenopea/MKTG6264-AIMLMktgAutomation/blob/main/Case_Study_3_Raw_Code_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem Statement

A casual dining chain in Columbus, OH that is known for its chicken sandwiches charges different prices at each of its restaurants and frequently changes the price of its sandwiches. In addition, each store has a coupon scheme whereby different value coupons are applicable at only certain restaurants (for example, someone would get a $1.25 coupon that is good only at the Italian Village location). ALmost all chicken sandwiches are sold using a coupon.

The resulting menu and marketing costs are high; and, while customers appreciate the quirkiness, there is anecdotal evidence that the unpredictability of prices drives customers away. Hence, the firm wants to investigate  whether it should standardize the cost of its sandwiches  at either the city-wide or the locational level (that is, make it uniform either within each restaurant or across all of the restaurants).

Specifically, your client wants to know :
a. whether it should unify the price of its chicken sandwiches across Columbus

b. whether it should unify the price at individual stores;

c. what the optimal price of the chicken sandwiches should be; and

d. whether it should continue to offer coupons, and, if so, what the optimal coupon should be.
This will involve a mixture of marketing theory and regression analysis.

Data
The client has given you data for January through April for all restaurants except one, which has February through April data. The data comprise the following variables:
- STORE : an indicator of the store

- MEDIAN_INC : the median income of the zipcode where the restaurant is located
- SOLD : how many chicken sandwiches were sold using the coupon
- PRICE : the menu price of a chicken sandwich at that store in that month
- DISCOUNT : the coupon discount given at that store for that month
- HAMBURGER : the price of a hamburger at the restaurant
- MENU_WA : the weighted average of all goods on the menu (weighted by volume sold).

You will work in groups. One or more members of your group will give a five-minute presentation on Wednesday, May 3. 

A final, actionable memo is due Friday, May 5, at 11:59 pm.

Only one member of your group needs to submit the meno online.
You will be graded on the actionable memo and the quality of your oral presentation.

### **Part 1. Data Prep**

In [23]:
import pandas as pd
import matplotlib.pyplot as plt

file_path = '/content/drive/MyDrive/DATA_STORE/Chicken_Columbus.csv'

df = pd.read_csv(file_path)


In [24]:
#sample 10 random records
df.sample(10)

,STORE,MEDIAN_INC,SOLD,PRICE,DISCOUNT,HAMBURGER,MENU_WA
0,HIGH_ST,30179,27856,425,50,80,231.6
5,ITAL_VIL,17876,33388,385,60,85,247.2
7,ITAL_VIL,17876,36418,380,65,85,254.4
8,COL_ST,33912,36723,385,60,90,256.2
21,BLOOR_ST,72099,51795,665,140,225,461.6
4,ITAL_VIL,17876,31280,375,50,80,237.4
10,COL_ST,33912,40430,390,70,110,286.6
18,BROAD_ST,41290,46793,640,130,170,364.2
12,GER_VIL,51020,41823,400,75,115,301.4
17,BROAD_ST,41290,44178,565,115,140,364.0


In [25]:
# calculate averages for each store since some stores are missing monthly data (this way, we can only normalize everything as numbers per month)
averages = df.groupby('STORE')['SOLD'].mean().reset_index()
averages.rename(columns={'SOLD':'mean_sold'}, inplace=True)

averages['mean_discount'] = df.groupby('STORE')['DISCOUNT'].mean().reset_index()['DISCOUNT']


averages['mean_med_inc'] = df.groupby('STORE')['MEDIAN_INC'].mean().reset_index()['MEDIAN_INC']
averages['mean_price'] = df.groupby('STORE')['PRICE'].mean().reset_index()['PRICE']
averages['mean_hamburger_price'] = df.groupby('STORE')['HAMBURGER'].mean().reset_index()['HAMBURGER']
averages['mean_menu_wa'] = df.groupby('STORE')['MENU_WA'].mean().reset_index()['MENU_WA']

#format all the columns to 2 decimal places
averages = averages.round(2)

averages

,STORE,mean_sold,mean_discount,mean_med_inc,mean_price,mean_hamburger_price,mean_menu_wa
0,BLOOR_ST,51364.50,138.75,72099.0,645.00,221.25,449.75
1,BROAD_ST,43462.00,122.50,41290.0,592.50,150.00,370.00
2,COL_ST,38545.33,66.67,33912.0,393.33,98.33,270.73
3,GER_VIL,40850.25,81.25,51020.0,453.75,118.75,309.60
4,HIGH_ST,29601.75,51.25,30179.0,402.50,80.00,235.05
5,ITAL_VIL,34172.50,60.00,17876.0,383.75,83.75,247.90


### **Part 2. Create regression model for the given dataset**

In [26]:
import statsmodels.api as sm

# define the target and predictor variables
y = df['SOLD']
X = df[['MEDIAN_INC', 'PRICE', 'DISCOUNT', 'HAMBURGER', 'MENU_WA']]

# add constant to predictor variables
X = sm.add_constant(X)

# fit and summarize the OLS (ordinary least squares) model


model = sm.OLS(y, X).fit()

predicted = model.fittedvalues
residuals = model.resid

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                   SOLD   R-squared:                       0.960
Model:                            OLS   Adj. R-squared:                  0.948
Method:                 Least Squares   F-statistic:                     80.92
Date:                Mon, 17 Apr 2023   Prob (F-statistic):           3.04e-11
Time:                        00:20:13   Log-Likelihood:                -200.04
No. Observations:                  23   AIC:                             412.1
Df Residuals:                      17   BIC:                             418.9
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        4.91e+04   7019.136      6.995      0.0

In [27]:
# determine the regression equation
target_var = 'SOLD'
predictor_vars = ['MEDIAN_INC', 'PRICE', 'DISCOUNT', 'HAMBURGER', 'MENU_WA']

X = df[predictor_vars]
y = df[target_var]
X = sm.add_constant(X)

model = sm.OLS(y, X).fit()

coefs = model.params
equation = f'SOLD = {coefs[0]:.2f}'
for i in range(1,len(predictor_vars)+1):
    coef_label = coefs.index[i]
    coef_val = coefs[i]
    equation += f' + {coef_val:.2f} * {coef_label}'

print(equation)

SOLD = 49099.86 + 0.13 * MEDIAN_INC + -83.86 * PRICE + 399.66 * DISCOUNT + 127.35 * HAMBURGER + -80.41 * MENU_WA


### **Part 3. Explaining regression output:**

We can see the impact of each variable on the model based on their coefficients. 

- `MEDIAN_INC`: For a one-unit increase in median income, the predicted value of SOLD will increase by 0.1284 units, holding other variables.

- `PRICE`: For a one-unit increase in price, the predicted value of SOLD will decrease by 83.8607 units, holding other variables constant.

- `DISCOUNT`: For a one-unit increase in discount, the predicted value of SOLD will increase by 399.6590 units, holding  other variables constant.

- `HAMBURGER`: For a one-unit increase in hamburger sales, the predicted value of SOLD will increase by 127.3455 units, holding other variables constant.

- `MENU_WA`: For a one-unit increase in menu advertisements, the predicted value of SOLD will decrease by 80.41 units, holding other variables constant.

Therefore based on the given regression coefficients, we can conclude that the most impactful feature in predicting SOLD is `DISCOUNT`, followed by the price of a `HAMBURGER` and `MEDIAN_INC`, which is median income on which the restaurant is located. On the other hand, PRICE has a negative impact on SOLD, while MENU_WA has relatively less impact.


### **Part 4. Initial Analysis**

Now that we know the regression coefficients and know that it has a relatively large R2 value, we can focus our strategy on key areas:

- Start by modifying `DISCOUNT` levels for each of the stores per month, keeping everything else constant. Determing the sensitivity for demand given the level of discount.

- Once sales volume hits it's peak or we see diminishing returns for a specified discount for a store, we can adjust hamburger prices for the same discount until demand falls and we see diminishing returns.

- For optimal performance, since we know that median_income is a correlated variable to the amount of chicken sandwiches sold, we can raise discounts and hamburger prices in higher increments in stores with higher median incomes than stores with lower median incomes

- Since there is a negative correlation with the volume of chicken sandwiches sold for its price, we should unify the price of chicken sandwiches across all stores. 

- Since there is a negative correlation with the weighted average of all goods on the menu, we should simplify them across all stores. We can start by normalizing the price based on the mean sales for a particular store.

- To get the optimal price of the chicken sandwich, we can use the regression line equation and the mean value for each of the stores

Step 1: 

`MEAN_SOLD_ALL_STORES = 49099.86 + 0.13 * MEAN_MEDIAN_INC - 83.86 * OPTIMAL_PRICE + 399.66 * MEAN_DISCOUNT + 127.35 * MEAN_HAMBURGER - 80.41 * MEAN_MENU_WA`

Step 2:

`83.86 * OPTIMAL_PRICE = 49099.86 + 0.13 * MEAN_MEDIAN_INC + 399.66 * MEAN_DISCOUNT + 127.35 * MEAN_HAMBURGER - 80.41 * MEAN_MENU_WA - MEAN_SOLD`

Step 3:


`OPTIMAL_PRICE = (48675.28 + 0.13 * MEAN_MEDIAN_INC + 399.66 * MEAN_DISCOUNT + 127.35 * MEAN_HAMBURGER - 80.41 * MEAN_MENU_WA) / 83.86`

In [28]:
# here are the monthly averages per each store

averages.set_index('STORE', inplace=True)
averages

,mean_sold,mean_discount,mean_med_inc,mean_price,mean_hamburger_price,mean_menu_wa
STORE,,,,,,
BLOOR_ST,51364.50,138.75,72099.0,645.00,221.25,449.75
BROAD_ST,43462.00,122.50,41290.0,592.50,150.00,370.00
COL_ST,38545.33,66.67,33912.0,393.33,98.33,270.73
GER_VIL,40850.25,81.25,51020.0,453.75,118.75,309.60
HIGH_ST,29601.75,51.25,30179.0,402.50,80.00,235.05
ITAL_VIL,34172.50,60.00,17876.0,383.75,83.75,247.90


In [29]:
# here are all the coefficients

const_coeff = coefs[0]
med_inc_coeff = coefs[1]
price_coeff = coefs[2]
discount_coeff = coefs[3]
hamburger_coeff = coefs[4]
menu_wa_coeff = coefs[5]

#print all coefficients
print('Constant Coefficient: ', const_coeff)
print('Median Income Coefficient: ', med_inc_coeff)
print('Price Coefficient: ', price_coeff)
print('Discount Coefficient: ', discount_coeff)
print('Hamburger Coefficient: ', hamburger_coeff)
print('Menu WA Coefficient: ', menu_wa_coeff)

Constant Coefficient:  49099.85578606592
Median Income Coefficient:  0.1283736177209982
Price Coefficient:  -83.860691155122
Discount Coefficient:  399.65896696697484
Hamburger Coefficient:  127.34548581335082
Menu WA Coefficient:  -80.4099909051804


In [30]:
# calculating the optimal price of a chicken sandwich and optimal discounts (coupon? for each store with our regression model

optimal_prices = {}
optimal_discounts = {}
for i in range(0, len(averages)):
    #save index name to variable store
    store = averages.index[i]
    med_inc = averages.iloc[i,2]
    price = averages.iloc[i,3]
    discount = averages.iloc[i,1]
    hamburger = averages.iloc[i,4]
    menu_wa = averages.iloc[i,5]
    mean_sold= averages.iloc[i,0]
    optimal_prices[store] = abs((const_coeff - mean_sold +
                              med_inc_coeff * med_inc + 
                              discount_coeff * discount + 
                              hamburger_coeff * hamburger + 
                              menu_wa_coeff * menu_wa)/price_coeff)
    optimal_discounts[store] = abs((const_coeff - mean_sold +
                              med_inc_coeff * med_inc + 
                              price_coeff * price + 
                              hamburger_coeff * hamburger + 
                              menu_wa_coeff * menu_wa)/discount_coeff)

#building a final table with averages per each store and all the corresponding optimal discounts and chicken sandwich prices
optimal_discounts = pd.DataFrame.from_dict(optimal_discounts, orient='index').reset_index()
optimal_discounts.rename(columns={'index':'store', 0:'optimal_discount'}, inplace=True)

optimal_prices = pd.DataFrame.from_dict(optimal_prices,orient='index').reset_index()
optimal_prices.rename(columns={'index':'store', 0:'optimal_price'}, inplace=True)

optimal_prices = pd.merge(optimal_prices, optimal_discounts, on='store')
optimal_prices.set_index('store', inplace=True)

averages = pd.merge(averages, optimal_prices,left_index=True, right_index=True)

In [31]:
averages

,mean_sold,mean_discount,mean_med_inc,mean_price,mean_hamburger_price,mean_menu_wa,optimal_price,optimal_discount
STORE,,,,,,,,
BLOOR_ST,51364.50,138.75,72099.0,645.00,221.25,449.75,649.344067,137.838482
BROAD_ST,43462.00,122.50,41290.0,592.50,150.00,370.00,587.244768,123.602709
COL_ST,38545.33,66.67,33912.0,393.33,98.33,270.73,385.230310,68.369563
GER_VIL,40850.25,81.25,51020.0,453.75,118.75,309.60,447.156607,82.633496
HIGH_ST,29601.75,51.25,30179.0,402.50,80.00,235.05,419.053138,47.776645
ITAL_VIL,34172.50,60.00,17876.0,383.75,83.75,247.90,380.789233,60.621260
